In [1]:
!pip uninstall -y torch
!pip install -q --no-index --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-vllm vllm
!pip install -q -U /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q -U /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q --no-deps --no-index /kaggle/input/hf-libraries/sentence-transformers/sentence_transformers-3.1.0-py3-none-any.whl
!pip install --no-deps --no-index /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Processing /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl


In [2]:
!pip install transformers peft accelerate \
    -q -U --no-index --find-links /kaggle/input/lmsys-wheel-files

In [3]:
!pip install --no-index /kaggle/input/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/optimum-1.21.2-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/auto_gptq-0.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --find-links=/kaggle/input/bitsandbytes0-42-0

Looking in links: /kaggle/input/bitsandbytes0-42-0
Processing /kaggle/input/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl
Looking in links: /kaggle/input/bitsandbytes0-42-0
Processing /kaggle/input/bitsandbytes0-42-0/optimum-1.21.2-py3-none-any.whl
Processing /kaggle/input/bitsandbytes0-42-0/coloredlogs-15.0.1-py2.py3-none-any.whl (from optimum==1.21.2)
Processing /kaggle/input/bitsandbytes0-42-0/transformers-4.42.4-py3-none-any.whl (from transformers[sentencepiece]<4.43.0,>=4.26.0->optimum==1.21.2)
Processing /kaggle/input/bitsandbytes0-42-0/tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from transformers<4.43.0,>=4.26.0->transformers[sentencepiece]<4.43.0,>=4.26.0->optimum==1.21.2)
Processing /kaggle/input/bitsandbytes0-42-0/humanfriendly-10.0-py2.py3-none-any.whl (from coloredlogs->optimum==1.21.2)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully unins

In [4]:
import gc
import os
import re
from typing import List

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
from numpy.linalg import norm
from peft import LoraConfig, get_peft_model
from sklearn.neighbors import NearestNeighbors
from torch import Tensor
from tqdm.autonotebook import trange
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoConfig, AutoTokenizer, AutoModelForMaskedLM, AutoModel, BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)

In [5]:
test_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
misconception_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

model_path = "/kaggle/input/qwen2.5-14/pytorch/default/1"
lora_path='/kaggle/input/qwen14b-it-lora/lora_weights/adapter.bin'

device='cuda:0'

In [6]:
IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))

In [7]:
rows = []
for idx, row in test_df.iterrows():
    for option in ["A", "B", "C", "D"]:
        if option == row.CorrectAnswer:
            continue

        correct_answer = row[f"Answer{row.CorrectAnswer}Text"]

        query_text = f"""
        SubjectName: {row['SubjectName']}
        ConstructName: {row['ConstructName']}
        Question: {row['QuestionText']}
        Correct Answer: {correct_answer}
        Misconcepte Incorrect answer: {option}.{row[f'Answer{option}Text']}
        """
        rows.append(
            {
                "query_text": query_text.strip(),
                "QuestionId_Answer": f"{row.QuestionId}_{option}",
                "ConstructName": row.ConstructName,
                "SubjectName": row.SubjectName,
                "QuestionText": row.QuestionText,
                "correct_answer": correct_answer,
                "incorrect_answer": row[f"Answer{option}Text"],
            }
        )

df = pd.DataFrame(rows)
df.head()

,query_text,QuestionId_Answer,ConstructName,SubjectName,QuestionText,correct_answer,incorrect_answer
0,SubjectName: BIDMAS\n ConstructName: Us...,1869_B,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \)
1,SubjectName: BIDMAS\n ConstructName: Us...,1869_C,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times(2+4-5) \)
2,SubjectName: BIDMAS\n ConstructName: Us...,1869_D,Use the order of operations to carry out calcu...,BIDMAS,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),Does not need brackets
3,SubjectName: Simplifying Algebraic Fractions\n...,1870_A,Simplify an algebraic fraction by factorising ...,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",Does not simplify,\( m+1 \)
4,SubjectName: Simplifying Algebraic Fractions\n...,1870_B,Simplify an algebraic fraction by factorising ...,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",Does not simplify,\( m+2 \)


In [8]:
# 移動張量
def batch_to_device(batch, target_device):
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

In [9]:
def last_token_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

In [10]:
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

In [11]:
def inference(df, model, tokenizer, device):
    batch_size = 16
    max_length = 512
    sentences = list(df['query_text'].values)

    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True, return_tensors="pt")
        features = batch_to_device(features, device)
        with torch.no_grad():
            outputs = model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    return np.concatenate(all_embeddings, axis=0)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(lora_path.replace("/adapter.bin",""))
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
model_base = AutoModel.from_pretrained(model_path, 
                                  quantization_config=bnb_config, 
                                  device_map=device,
                                  trust_remote_code=True)

if lora_path:
    print("loading lora")
    config = LoraConfig(
        r=64,
        lora_alpha=128,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        bias="none",
        lora_dropout=0.05,  # Conventional
        task_type="FEATURE_EXTRACTION",
    )
    model = get_peft_model(model_base, config)
    d = torch.load(lora_path, map_location=model.device)
    model.load_state_dict(d, strict=False)
    model = model.merge_and_unload()
model = model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

loading lora


/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [13]:
task_description = 'Given a math question with correct answer and a misconcepted incorrect answer, retrieve the most accurate misconception for the incorrect answer.'
df['query_text'] = df['query_text'].apply(lambda x: get_detailed_instruct(task_description, x))

V_answer = inference(df, model, tokenizer, device)

misconception_df["query_text"] = misconception_df["MisconceptionName"]
if not IS_SUBMISSION:
    misconception_df = misconception_df[:100]


V_misconception = inference(misconception_df, model, tokenizer, device)

V_answer.shape,V_misconception.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

((9, 5120), (100, 5120))

In [14]:
def get_matches(V_topic, V_content, n_neighbors=25):
    neighbors_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm="brute", n_jobs=-1)
    neighbors_model.fit(V_content)
    dists, indices = neighbors_model.kneighbors(V_topic)
    
    return indices

In [15]:
indices = get_matches(V_answer, V_misconception, n_neighbors=25)
indices.shape

(9, 25)

In [16]:
del model_base, model, tokenizer

gc.collect()
torch.cuda.empty_cache()

In [17]:
np.save("indices.npy", indices)
df.to_parquet("df.parquet", index=False)

In [18]:
%%writefile run_vllm.py

import vllm
import numpy as np
import pandas as pd
from transformers import PreTrainedTokenizer, AutoTokenizer
from typing import List
import torch
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
import re
from copy import deepcopy

import logging
from transformers.utils.logging import disable_progress_bar

# 調整 vLLM 的日誌
disable_progress_bar()
logging.getLogger("vllm").setLevel(logging.CRITICAL)
logging.getLogger("transformers").setLevel(logging.CRITICAL)
logging.getLogger("torch").setLevel(logging.CRITICAL)
logging.getLogger("vllm.triton_utils").setLevel(logging.CRITICAL)

# load anything
model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"
tokenizer = AutoTokenizer.from_pretrained(model_path)
df = pd.read_parquet("df.parquet")
misconception_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")
indices = np.load("indices.npy")

print("indices :")
print(indices)

# prompt
PROMPT  = """Here is a question about {ConstructName}({SubjectName}).
Question: {Question}
Correct Answer: {CorrectAnswer}
Incorrect Answer: {IncorrectAnswer}

You are a Mathematics teacher. Your task is to reason and identify the misconception behind the Incorrect Answer with the Question.
Answer concisely what misconception it is to lead to getting the incorrect answer.
Pick the correct misconception number from the below:

{Retrival}
"""


SYSTEM_PROMPT = """You are an expert Mathematics teacher specialized in diagnosing and addressing students' conceptual misunderstandings. Your job is to analyze their incorrect answers, trace the reasoning error, and identify the precise underlying misconception from the provided options."""


# preprocess function
def preprocess_text(x):
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

# chat message
def apply_template(row, tokenizer):
    messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT
        },
        {
            "role": "user", 
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"incorrect_answer"],
                    CorrectAnswer=row[f"correct_answer"],
                    Retrival=row[f"retrieval"]
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    # print(messages[1]["content"])
    return text

# vllm config
llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.90, 
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True
)
tokenizer = llm.get_tokenizer()

# process candidate
def get_candidates(c_indices):
    candidates = []

    mis_names = misconception_df["MisconceptionName"].values
    for ix in c_indices:
        c_names = []
        for i, name in enumerate(mis_names[ix]):
            c_names.append(f"{i+1}. {name}")

        candidates.append("\n".join(c_names))
        
    return candidates

# spilt group
def split_and_concatenate(array1, group_size):
    array1 = array1[:,:-1]
    m, n = array1.shape
    
    num_groups = -(-n // group_size)
    results = []

    # 倒序處理 array1 的分組
    for i in range(num_groups):
        start_col = n - (i + 1) * group_size
        end_col = n - i * group_size

        if start_col < 0:
            start_col = 0

        group = array1[:, start_col:end_col]

        results.append(group)  # 插入到最前方以保留順序

    return results

# filter survivors
def filter(remain_indices, survivors):

    survivors = survivors.flatten()
    
    mask = remain_indices != survivors[:, np.newaxis]

    filtered_remain_indices = [row[mask[i]] for i, row in enumerate(remain_indices)]
    
    filtered_remain_indices = np.array(filtered_remain_indices)
    
    return filtered_remain_indices


# main function
remain_indices = deepcopy(indices)
all_survivors = []
groups = 8

for i in range(3):
    survivors = remain_indices[:, -1:]
    c_indices = split_and_concatenate(remain_indices, groups)

    for j in range(len(c_indices)):
        group = c_indices[j]
        group = np.hstack((group, survivors))
        choices = [str(i) for i in range(1, len(group[0]) + 1)]

        print(f"{i}:{j} - {group}")


        df["retrieval"] = get_candidates(group)
        df["text"] = df.apply(lambda row: apply_template(row, tokenizer), axis=1)

        # print(df["text"].values[0])
        # print()

        responses = llm.generate(
            df["text"].values,
            vllm.SamplingParams(
                n=1,  # Number of output sequences to return for each prompt.
                top_k=1,  # Float that controls the cumulative probability of the top tokens to consider.
                temperature=0,  # randomness of the sampling
                seed=777, # Seed for reprodicibility
                skip_special_tokens=False,  # Whether to skip special tokens in the output.
                max_tokens=1,  # Maximum number of tokens to generate per output sequence.
                logits_processors=[MultipleChoiceLogitsProcessor(tokenizer, choices=choices)]
            ),
            use_tqdm=False
        )
        
        responses = [x.outputs[0].text for x in responses]
        df["response"] = responses

        llm_choices = df["response"].astype(int).values - 1
        print(llm_choices)

        survivors = np.array([cix[best] for best, cix in zip(llm_choices, group)]).reshape(-1, 1)
        print(f"{i}:{j} - {survivors}")

    remain_indices = filter(remain_indices, survivors)
    all_survivors.append(survivors)
    
all_survivors = np.hstack(all_survivors)
print("all survivors")
print(all_survivors)

# save file
results = []

for i in range(indices.shape[0]):
    temp = [str(x) for x in all_survivors[i]] + [
        str(x) for x in indices[i] if x not in all_survivors[i]
    ]
    temp = temp[:25]
    results.append(" ".join(temp))

df["MisconceptionId"] = results
df.to_csv("submission.csv", columns=["QuestionId_Answer", "MisconceptionId"], index=False)

Writing run_vllm.py


In [19]:
!export NCCL_DEBUG=CRITICAL
!export LOGLEVEL=CRITICAL
!export TOKENIZERS_PARALLELISM=false
!python run_vllm.py

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid de

indices :
[[15 55 77 95 82 96 34  4 13 75 74 17 29 42 73 14 38 84 60  8 24 89 30 25
  52]
 [77 15 55 82 13 74 14 25 29 84 52 30 38 73 98  0 42 24 68 51 95  9 71 49
  23]
 [15 77 74 55 95 14 13 24 29 73 82  0 25 52  4 96 38 17  3 84 90 89 80 64
  34]
 [59 80 91  3 29 55 46 79 82 34 74  1 94  6 19  0 44 24 96 58 32 20 49 25
  98]
 [59 80 91  3 79 46 29 74 55 34 58 94  6 27 14 24 32 73 44 49 82 65 68  1
  15]
 [59 80 91  3 29 46 79 55 82 74  1 24 98  6 34 25  0 49 14 73 68 65 76 58
  94]
 [14 25 82 38 30 49 74 23 24 36  0 56 84 55 37 63 97 85 98 62 79 18 46  7
  80]
 [25 82 14 49 38  0 74 62 85 56 24 30 55 36 84 23 97 18 37  7  8  3 40 75
  31]
 [ 0 23 56 14 25 18 62 74 84 24  3 38 85 82 49 87 97 31 95 55 79 89 12 63
   7]]
Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:19<01:17, 19.49s/it]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:40<01:01, 20.34s/it]
Loading safetensors ch

In [20]:
pd.read_csv("submission.csv")

,QuestionId_Answer,MisconceptionId
0,1869_B,74 77 15 55 95 82 96 34 4 13 75 17 29 42 73 14...
1,1869_C,15 14 13 77 55 82 74 25 29 84 52 30 38 73 98 0...
2,1869_D,15 77 14 74 55 95 13 24 29 73 82 0 25 52 4 96 ...
3,1870_A,59 3 29 80 91 55 46 79 82 34 74 1 94 6 19 0 44...
4,1870_B,59 79 46 80 91 3 29 74 55 34 58 94 6 27 14 24 ...
5,1870_C,59 3 79 80 91 29 46 55 82 74 1 24 98 6 34 25 0...
6,1871_A,38 30 23 14 25 82 49 74 24 36 0 56 84 55 37 63...
7,1871_C,25 38 49 82 14 0 74 62 85 56 24 30 55 36 84 23...
8,1871_D,14 25 18 0 23 56 62 74 84 24 3 38 85 82 49 87 ...


In [21]:
# %%writefile run_vllm2.py

# import vllm
# import numpy as np
# import pandas as pd
# from transformers import PreTrainedTokenizer, AutoTokenizer
# from typing import List
# import torch
# from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
# import re

# import logging
# from transformers.utils.logging import disable_progress_bar

# # 調整 vLLM 的日誌
# disable_progress_bar()
# logging.getLogger("vllm").setLevel(logging.CRITICAL)
# logging.getLogger("transformers").setLevel(logging.CRITICAL)
# logging.getLogger("torch").setLevel(logging.CRITICAL)
# logging.getLogger("vllm.triton_utils").setLevel(logging.CRITICAL)


# model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"
# tokenizer = AutoTokenizer.from_pretrained(model_path)


# def preprocess_text(x):
#     x = re.sub("http\w+", '',x)   # Delete URL
#     x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
#     x = re.sub(r"\,+", ",", x)
#     x = re.sub(r"\\\(", " ", x)
#     x = re.sub(r"\\\)", " ", x)
#     x = re.sub(r"[ ]{1,}", " ", x)
#     x = x.strip()                 # Remove empty characters at the beginning and end
#     return x

# PROMPT  = """Here is a question about {ConstructName}({SubjectName}).
# Question: {Question}
# Correct Answer: {CorrectAnswer}
# Incorrect Answer: {IncorrectAnswer}

# You are a Mathematics teacher. Your task is to reason and identify the misconception behind the Incorrect Answer with the Question.
# Answer concisely what misconception it is to lead to getting the incorrect answer.
# Pick the correct misconception number from the below:

# {Retrival}
# """
# # just directly give your answers.

# def apply_template(row, tokenizer):
#     messages = [
#         {
#             "role": "user", 
#             "content": preprocess_text(
#                 PROMPT.format(
#                     ConstructName=row["ConstructName"],
#                     SubjectName=row["SubjectName"],
#                     Question=row["QuestionText"],
#                     IncorrectAnswer=row[f"incorrect_answer"],
#                     CorrectAnswer=row[f"correct_answer"],
#                     Retrival=row[f"retrieval"]
#                 )
#             )
#         }
#     ]
#     text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     return text


# misconception_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

# df = pd.read_parquet("df.parquet")
# indices = np.load("indices.npy")

# print(indices)

# model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"

# llm = vllm.LLM(
#     model_path,
#     quantization="awq",
#     tensor_parallel_size=2,
#     gpu_memory_utilization=0.90, 
#     trust_remote_code=True,
#     dtype="half", 
#     enforce_eager=True,
#     max_model_len=5120,
#     disable_log_stats=True
# )
# tokenizer = llm.get_tokenizer()


# def get_candidates(c_indices):
#     candidates = []

#     mis_names = misconception_df["MisconceptionName"].values
#     for ix in c_indices:
#         c_names = []
#         for i, name in enumerate(mis_names[ix]):
#             c_names.append(f"{i+1}. {name}")

#         candidates.append("\n".join(c_names))
        
#     return candidates

# survivors = indices[:, -1:]

# for i in range(3):
#     c_indices = np.concatenate([indices[:, -8*(i+1)-1:-8*i-1], survivors], axis=1)

#     print(f"{i} {c_indices}")
    
#     df["retrieval"] = get_candidates(c_indices)
#     df["text"] = df.apply(lambda row: apply_template(row, tokenizer), axis=1)
    
#     # print("Example:")
#     # print(df["text"].values[0])
#     # print()
    
#     responses = llm.generate(
#         df["text"].values,
#         vllm.SamplingParams(
#             n=1,  # Number of output sequences to return for each prompt.
#             top_k=1,  # Float that controls the cumulative probability of the top tokens to consider.
#             temperature=0,  # randomness of the sampling
#             seed=777, # Seed for reprodicibility
#             skip_special_tokens=False,  # Whether to skip special tokens in the output.
#             max_tokens=1,  # Maximum number of tokens to generate per output sequence.
#             logits_processors=[MultipleChoiceLogitsProcessor(tokenizer, choices=["1", "2", "3", "4", "5", "6", "7", "8", "9"])]
#         ),
#         use_tqdm=True
#     )
    
#     responses = [x.outputs[0].text for x in responses]
#     df["response"] = responses
    
    
#     llm_choices = df["response"].astype(int).values - 1
    
#     survivors = np.array([cix[best] for best, cix in zip(llm_choices, c_indices)]).reshape(-1, 1)
#     print(survivors)



# results = []

# for i in range(indices.shape[0]):
#     ix = indices[i]
#     llm_choice = survivors[i, 0]
    
#     results.append(" ".join([str(llm_choice)] + [str(x) for x in ix if x != llm_choice]))


# df["MisconceptionId"] = results
# df.to_csv("submission2.csv", columns=["QuestionId_Answer", "MisconceptionId"], index=False)

In [22]:
# !export NCCL_DEBUG=CRITICAL
# !export LOGLEVEL=CRITICAL
# !export TOKENIZERS_PARALLELISM=false
# !python run_vllm2.py

In [23]:
# pd.read_csv("submission2.csv")